<h1>LangGraph ReAct Agent for AI Avatar Video Generation</h1>

In [ ]:
%pip install langgraph python-dotenv nest_asyncio

In [ ]:
pip install --upgrade langgraph

In [ ]:
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import requests
import os
import base64
import aiohttp
import asyncio
import time

from dotenv import load_dotenv
_ = load_dotenv()

import nest_asyncio
nest_asyncio.apply()

DID_API_KEY = os.environ["DID_API_KEY"]


In [ ]:
# Tavily Search API
from tavily import TavilyClient
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def search_tavily(query):
    """Search for supporting sources using Tavily."""
    response = tavily.search(query=query, max_results=2)
    return response

# search_tavily("how many different cat breeds are there")

search_tool = Tool(
    name="TavilySearch",
    func=search_tavily,
    description="Finds relevant sources of information online."
)

In [ ]:
# Initialize the LLM model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
def generate_script(topic):
    """Generate a structured script for the video and return only the script text."""
    
    prompt = f"""
    Generate a clear, concise educational script for a video on: {topic}.
    Return ONLY the script as plain text. Do NOT include a title, JSON formatting, explanations, or extra words.
    """

    response = llm.invoke(prompt)
    
    try:
        script_text = response.content.strip()  # Remove leading/trailing whitespace

        # Handle cases where the model wraps the response in markdown-style JSON formatting
        if script_text.startswith("```json"):
            script_text = script_text.strip("```json").strip("```").strip()
            script_text = json.loads(script_text).get("content", script_text)  # Extract content if JSON

        return script_text  # Return only the script text
    except Exception as e:
        raise ValueError(f"Failed to parse script output: {response.content}. Error: {e}")



script_tool = Tool(
    name="ScriptGenerator",
    func=generate_script,
    description="Generates a structured script for the video. Returns only the script text as a string."
)



In [ ]:
# Generate Talking Avatar using D-ID API from a Custom Image
async def generate_avatar(script_text):
    """Asynchronously generate a talking avatar video using D-ID API."""

    # Validate API Key format
    if not DID_API_KEY or ":" not in DID_API_KEY:
        raise ValueError("❌ ERROR: Invalid API Key format. Basic Authentication requires 'username:password'.")

    username, password = DID_API_KEY.split(":", 1)
    encoded_auth = base64.b64encode(f"{username}:{password}".encode()).decode()

    headers = {
        "Authorization": f"Basic {encoded_auth}",
        "Content-Type": "application/json"
    }

    payload = {
        "source_url": "https://raw.githubusercontent.com/jenyss/AIAvatarAgent/main/Eli_cat.jpeg",
        "script": {
            "type": "text",
            "provider": {
                "type": "microsoft",
                "voice_id": "en-GB-MaisieNeural"
            },
            "input": script_text
        },
    }

    async with aiohttp.ClientSession() as session:
        try:
            print("🚀 Sending request to D-ID API...")
            async with session.post("https://api.d-id.com/talks", json=payload, headers=headers) as response:
                response_json = await response.json()
                print("🔎 API Response:", response_json)

                if response.status not in [200, 201]:  # Accept 201 as valid!!!
                    raise ValueError(f"❌ ERROR: Failed to generate avatar. HTTP Status: {response.status}")

                request_id = response_json.get("id")
                if not request_id:
                    raise ValueError("❌ ERROR: D-ID API did not return a request ID.")

                print(f"✅ SUCCESS: Avatar generation request submitted! Request ID: {request_id}")

                # Call the function to fetch the result
                await asyncio.sleep(25)  # Give it 25 seconds before checking
                return await get_avatar_result(request_id, session, headers)

        except aiohttp.ClientError as e:
            raise ValueError(f"❌ ERROR: Network or API issue - {str(e)}")

async def get_avatar_result(request_id, session, headers):
    """Asynchronously fetch the status of the generated avatar video from D-ID API."""

    status_url = f"https://api.d-id.com/talks/{request_id}"
    print("🔍 Checking avatar status...")

    while True:
        try:
            async with session.get(status_url, headers=headers) as response:
                response_json = await response.json()
                print("🔎 API Response:", response_json)

                if response.status not in [200, 201]:  # Accept 201 as valid!!!
                    raise ValueError(f"❌ ERROR: Failed to generate avatar. HTTP Status: {response.status}")


                status = response_json.get("status")
                if status == "done":
                    video_url = response_json.get("result_url")
                    print(f"✅ SUCCESS: Avatar generation complete! 🎬 Video URL: {video_url}")
                    return video_url
                elif status in ["failed", "error"]:
                    raise ValueError("❌ ERROR: Avatar generation failed.")

                print("⏳ Waiting for processing... Retrying in 10 seconds.")
                await asyncio.sleep(10)  # Wait 10 seconds before retrying

        except aiohttp.ClientError as e:
            raise ValueError(f"❌ ERROR: Network or API issue - {str(e)}")

avatar_tool = Tool(
    name="DIDAvatarGenerator",
    func=lambda script_text: asyncio.run(generate_avatar(script_text)),  # Ensures async execution
    coroutine=generate_avatar,  # Explicitly register it as async
    description="Generates a talking video avatar from a text script."
)


def download_avatar(url, output_dir="downloads"):
    """Download the generated avatar video from D-ID API and save it locally."""
    
    # Ensure the directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    output_path = os.path.join(output_dir, "avatar_video.mp4")
    
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error for HTTP request failures

        # Save file in chunks to avoid memory issues
        with open(output_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        return f"✅ Download complete: {output_path}"
    
    except requests.exceptions.RequestException as e:
        return f"❌ Download failed: {str(e)}"
    except FileNotFoundError as e:
        return f"❌ File system error: {str(e)}"


download_tool = Tool(
    name="AvatarDownloader",
    func=download_avatar,
    description="Downloads the generated avatar video from a given URL and saves it locally."
)

In [ ]:
# Define the tools for the agent
tools = [search_tool, script_tool, avatar_tool, download_tool]

system_prompt = "You are an AI agent that generates educational video content from a single text input."

# Create the LangGraph React Agent
graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt=system_prompt
)


In [ ]:
def generate_ai_video(user_input):
    """Run the LangGraph agent to generate a full AI-powered video."""
    inputs = {"messages": [("user", user_input)]}

    for step in graph.stream(inputs, stream_mode="values"):
        message = step["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

# Example Usage:
# generate_ai_video("Explain the history of AI in 60 seconds.")
# generate_ai_video("Tell about Albert Einstein in 50 seconds.")
generate_ai_video("Tell me how to grow tomatoes in 50 seconds. Download the video.")